In [39]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

# Generar los DataFrames

In [40]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [41]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [42]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

productos_sin_compra['tolerance_order_date'] = productos_sin_compra['order_date']

In [43]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [44]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [45]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [46]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [47]:
compras_odoo = pd.merge(
                    compras_linea,
                    compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'
                )

compras_odoo['tolerance_order_date'] = compras_odoo['order_date'].dt.normalize() - pd.Timedelta(days=5)

cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras_odoo[cols_to_Int64] = compras_odoo[cols_to_Int64].astype('Int64')

compras_odoo['product_qty'] = compras_odoo['product_qty'].astype('Float64')
compras_odoo['vendedora'] = compras_odoo['vendedora'].convert_dtypes()

compras = pd.concat([compras_odoo, productos_sin_compra])

# Checks

In [48]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Todo correcto con check1


In [49]:
check_costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        direction = 'backward')
)

check2 = check_costo_venta[check_costo_venta['product_price'].isna()][['product_id', 'product_name_x']]

print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 24 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10282,28586,Anticipo
13415,28638,Servicios de Facturación
18005,4,Anticipo (PdV)
65138,29663,Toallas Multiusos Scott 10Rollos *5630*
65406,29660,"Pinza Para Chofer 8"" Urrea 278G *5609*"
65454,29664,Guantes Trup Látex Rugoso Unitalla #14248 *5634*


# Preliminares... obtener los segmentos a trabajar

Sacar los productos que se compran con varios proveedores

In [50]:
prod_comprados_odoo = (compras_odoo[['partner_id', 'partner_name', 'product_id_pp', 'product_name']]
                    .sort_values('partner_id').groupby('product_id_pp')
                    .agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
                    ).reset_index()

prod_comprados_odoo['diff'] = prod_comprados_odoo[('partner_id', 'mean')]  == prod_comprados_odoo[('partner_id', 'first')]

ids_productos_varios_prov = prod_comprados_odoo.loc[prod_comprados_odoo['diff'] == False]['product_id_pp']
ids_productos_unicos_prov = prod_comprados_odoo.loc[prod_comprados_odoo['diff'] == True]['product_id_pp']

print(f"Se han comprado un total de {len(prod_comprados_odoo['product_id_pp'])} productos en Odoo:")
print(f'    - {len(ids_productos_varios_prov)} productos con más de un proveedor', f"{len(ids_productos_varios_prov)/len(prod_comprados_odoo['product_id_pp'])*100:.2f}%")
print(f'    - {len(ids_productos_unicos_prov)} productos con un único proveedor', f"{len(ids_productos_unicos_prov)/len(prod_comprados_odoo['product_id_pp'])*100:.2f}%")

Se han comprado un total de 5079 productos en Odoo:
    - 221 productos con más de un proveedor 4.35%
    - 4858 productos con un único proveedor 95.65%


De los productos comprados a más de un proveedor, se obtienen los que no han cambiado su costo en el tiempo

In [51]:
compras_prod_varios_prov = compras_odoo[compras_odoo['product_id_pp'].isin(ids_productos_varios_prov)]


compras_prod_varios_prov_agrupado = compras_prod_varios_prov.groupby('product_id_pp').agg({'product_name':['first'], 'line_id':['count'], 'product_price':['first', 'mean']}).reset_index()
compras_prod_varios_prov_agrupado['diff'] = compras_prod_varios_prov_agrupado[('product_price', 'first')] == compras_prod_varios_prov_agrupado[('product_price', 'mean')]

ids_prod_varios_prov_costo_sin_cambios = compras_prod_varios_prov_agrupado.loc[compras_prod_varios_prov_agrupado['diff'] == True]['product_id_pp']
ids_prod_varios_prov_costo_con_cambios = compras_prod_varios_prov_agrupado.loc[compras_prod_varios_prov_agrupado['diff'] == False]['product_id_pp']


print(f'De los {len(ids_productos_varios_prov)} productos comprados con varios proveedores:')
print(f'    - {len(ids_prod_varios_prov_costo_sin_cambios)} productos no han cambiado su costo, representan a un {len(ids_prod_varios_prov_costo_sin_cambios)/len(ids_productos_varios_prov)*100:.2f}%')
print(f'    - {len(ids_prod_varios_prov_costo_con_cambios)} productos sí han cambiado su costo, representan a un {len(ids_prod_varios_prov_costo_con_cambios)/len(ids_productos_varios_prov)*100:.2f}%')


De los 221 productos comprados con varios proveedores:
    - 5 productos no han cambiado su costo, representan a un 2.26%
    - 216 productos sí han cambiado su costo, representan a un 97.74%


### Productos fase A del costo de ventas

Se concatena los productos comprados a un único proveedor ('ids_productos_unicos_prov') y los productos comprados a varios proveedores que no han cambiado su precio ('ids_prod_varios_prov_costo_sin_cambios')

In [52]:
ids_prod_fase_A = pd.concat([ids_productos_unicos_prov, ids_prod_varios_prov_costo_sin_cambios]).sort_values()

### Productos fase B del costo de ventas

Resto de los productos comprados a varios proveedores, es decir, los que sí han cambiado su precio ('ids_prod_varios_prov_costo_con_cambios')

In [53]:
ids_prod_fase_B = ids_prod_varios_prov_costo_con_cambios

In [54]:
# Borrar lína: sólo para explicar

ventas_productos_varios_prov = ventas_año[ventas_año['product_id'].isin(ids_productos_varios_prov)]
ventas_productos_unico_prov = ventas_año[~ventas_año['product_id'].isin(ids_productos_varios_prov)]

compras_productos_varios_prov = compras[compras['product_id_pp'].isin(ids_productos_varios_prov)]
compras_productos_unico_prov = compras[~compras['product_id_pp'].isin(ids_productos_varios_prov)]

print('Líneas de venta:')
print(f'    {len(ventas_productos_varios_prov)} de productos comprados con varios proveedores', f'{len(ventas_productos_varios_prov)/len(ventas_año)*100:.2f}%')
print(f'    {len(ventas_productos_unico_prov)} de productos comprados con un único proveedores', f'{len(ventas_productos_unico_prov)/len(ventas_año)*100:.2f}%')

print('\nLíneas de compra:')
print(f'    {len(compras_productos_varios_prov)} de productos comprados con varios proveedores', f'{len(compras_productos_varios_prov)/len(compras)*100:.2f}%')
print(f'    {len(compras_productos_unico_prov)} de productos comprado con un único proveedores', f'{len(compras_productos_unico_prov)/len(compras)*100:.2f}%')

Líneas de venta:
    4282 de productos comprados con varios proveedores 6.54%
    61173 de productos comprados con un único proveedores 93.46%

Líneas de compra:
    1498 de productos comprados con varios proveedores 7.32%
    18956 de productos comprado con un único proveedores 92.68%


En el dataframe de compras de productos con varios proveedores, obtener el segmento de productos donde no cambia el precio (aunque sí cambie el proveedor) y el segmento donde sí cambie.

In [55]:
# Borrar lína: es sólo para explicar que porcentaje de productos con proveedor único han mantenido el precio durante el tiempo
a = compras_productos_unico_prov.sort_values(by='product_price').groupby('product_id_pp').agg({'product_name':['first'], 'line_id':['count'], 'product_price':['first', 'mean']})
a['wep'] = a[('product_price', 'first')] == a[('product_price', 'mean')]

print(f"De los {len(compras_productos_unico_prov)} productos con un único proveedor, {len(a.loc[a['wep'] == True])} no han variado su precio =", f"{len(a.loc[a['wep'] == True])/len(a)*100:.2f}%")
a.loc[a['wep'] == True]

De los 18956 productos con un único proveedor, 8125 no han variado su precio = 82.73%


product_name line_id  \
                                                           first   count   
product_id_pp                                                              
7381           [90104] Encendedor 50P YA-0093 Sayer Lack (50p...       1   
7384           Carretilla Con Zoclo Alto Negra #2095 Herralum...       0   
7389           [14704] Cono Colador Sayer YA-0086 Desechable ...       1   
7390                       Manguera Flexible Tipo T 648P *25341*       0   
7393           [90103] Lapiz P/Carpintero YA-0099 Sayer (pz) ...       1   
...                                                          ...     ...   
29657          [5592] Caja De Distribución 3 Barras 8 Servici...       1   
29658          [5593] Fotocelda Hasta 100w  Led O Fluorescent...       1   
29659          [5606] Soporte metálico para aire acondicionad...       1   
29661          [5623] Clavo P/Clav Neum 1 3/16" C-18 Trup #18...       1   
29662          [5626] Grillete de Acero 5/8" #44972 Truper *5...       1   

              product_price            wep  
                      first     mean        
product_id_pp                               
7381                   0.00     0.00  True  
7384                  11.46    11.46  True  
7389                   2.50     2.50  True  
7390                   0.01     0.01  True  
7393                   0.00     0.00  True  
...                     ...      ...   ...  
29657              1,204.31 1,204.31  True  
29658                 64.66    64.66  True  
29659                362.07   362.07  True  
29661                 89.22    89.22  True  
29662                 61.29    61.29  True  

[8125 rows x 5 columns]

In [56]:
# Borrar lína: es sólo para explicar por qué decido llevarlos a merge_asof dirección "backward"
a = compras_productos_unico_prov.sort_values(by='product_price').groupby('product_id_pp').agg({'product_name':['first'], 'line_id':['count'], 'product_price':['first', 'mean']})
a['wep'] = a[('product_price', 'first')] == a[('product_price', 'mean')]
a['wep2'] = a[('product_price', 'mean')] / a[('product_price', 'first')]

print(f"De los {len(compras_productos_unico_prov)} productos con un único proveedor, {len(a.loc[a['wep'] == False])} sí han variado su precio =", f"{len(a.loc[a['wep'] == False])/len(a)*100:.2f}%")
a.loc[a['wep'] == False].sort_values(by='wep2', ascending=False)

De los 18956 productos con un único proveedor, 1683 sí han variado su precio = 17.14%


product_name line_id  \
                                                           first   count   
product_id_pp                                                              
7388           [20457] Sudadera Ejetutiva S (1 PZ) YA-0001 *2...       2   
7391           [90110] Flexometro Sayer Lack 6P YA-0094 (6 pz...       2   
28075                          [188] SOGA SEDA 1" Trenzada *188*       1   
10585           [87411] Guante Neo. Grandes Truper 14271 *87411*       9   
28929          [2973] Gaveta Apilable De Plástico 41x31x18 cm...       6   
...                                                          ...     ...   
8596                      [10900] Rondana Plana 1" C#109 *10900*       2   
28751          [2497] Cepillo Para Impermeabilizar Cerdas de ...       3   
12341                        Varilla Roscada 5/8" (100Cm) *5423*       2   
8301                         [3053] Lija D/Agua Fan. #320 *3053*      21   
29609          [5438] Polipasto de cadena de 1Ton #16824 Trup...       3   

              product_price             wep  wep2  
                      first     mean               
product_id_pp                                      
7388                   0.00   215.00  False   inf  
7391                   0.00    76.49  False   inf  
28075                 74.03 6,232.02  False 84.18  
10585                  2.54    41.68  False 16.41  
28929                  4.75    55.56  False 11.70  
...                     ...      ...    ...   ...  
8596                  10.70    10.70  False  1.00  
28751                 53.92    53.92  False  1.00  
12341                125.34   125.34  False  1.00  
8301                   6.83     6.83  False  1.00  
29609              1,590.52 1,590.52  False  1.00  

[1683 rows x 6 columns]

# Fase A: punto focal "Líneas de venta"
## Productos comprados con un único proveedor y
## Productos comprados a varios proveedores que no cambiaron su precio en el tiempo

Se obtiene el costo de venta en el dataframe 'ventas_año' donde  los renglones de venta coinciden con los 'ids_prod_fase_A', su contraparte es el total de las compras.
Estas ventas no son de tipo "especial", sino venta normal y se trata de encontrar un "Último costo". Es por esto que no se usa tolerancia y la dirección es "backward".

In [57]:
costo_venta_fase_A = (
    pd.merge_asof(
        left = ventas_año.loc[ventas_año['product_id'].isin(ids_prod_fase_A)].sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        direction = 'backward')
)

In [58]:
len(ventas_año.loc[ventas_año['product_id'].isin(ids_prod_fase_A)].sort_values('invoice_date'))

56581

In [66]:
len(costo_venta_fase_A)

56581

# Fase B: punto focal "Líneas de compra"
## Productos comprados a varios proveedores que sí cambiaron su precio en el tiempo

### 1er vuelta: productos que con un match sencillo, coinciden en 3 condiciones [producto, vendedora, cantidad]

Se estudian las líneas de compras con 'ids_prod_fase_)

In [60]:
compras_fase_b = compras_odoo.loc[compras_odoo['product_id_pp'].isin(ids_prod_fase_B)]
print(f'Son un total de {len(compras_odoo)} líneas de compra en Odoo. No se están contando las líneas de costo inicial del SAE.')

print(f'\nEn la fase B, quedan sólo {len(compras_fase_b)} de ellas {len(compras_fase_b)/len(compras_odoo)*100:.2f}%')

Son un total de 13084 líneas de compra en Odoo. No se están contando las líneas de costo inicial del SAE.

En la fase B, quedan sólo 1411 de ellas 10.78%


Debido a que el punto focal son las compras, no se puede hablar de un costo de la venta, sino de un "match entre ids".  
Se obtiene el match de líneas de compra de fase B donde  los renglones de coinciden con los 'ids_productos_varios_prov_costo_con_cambios'. Su contraparte es el total de ventas_año.  
Debido a que estas compras son para ventas de tipo "Especial", se trata de buscar una venta exacta con las 3 condiciones descritas anteriormente. Se usa una tolerancia de 5 días, con dirección "nearest".

In [61]:
match_1ro = (
    pd.merge_asof(
        left = compras_fase_b.sort_values('order_date'), 
        right = ventas_año.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)

Se descartan del merge anterior los match donde hay líneas de compras unidos a líneas de venta repetidas

In [62]:
ids_fact_line_repetidas = match_1ro.loc[(~match_1ro['fact_line_id'].isna()) & (match_1ro['fact_line_id'].duplicated()), 'fact_line_id']
ids_order_line_con_fact_line_repetidas = match_1ro.loc[match_1ro['fact_line_id'].isin(ids_fact_line_repetidas), 'line_id']

print(f'Son {len(ids_order_line_con_fact_line_repetidas)} líneas de compras que se unieron a líneas repetidad de fact_line_id')


Son 25 líneas de compras que se unieron a líneas repetidad de fact_line_id


Segmentas del merge 'match_1ro_fase_b' los match que se pueden usar, delimitas el trabajo faltante

In [63]:
match_1ro_compras_ok = match_1ro.loc[(~match_1ro['fact_line_id'].isna()) & (~match_1ro['line_id'].isin(ids_order_line_con_fact_line_repetidas)), ['fact_line_id', 'line_id']]
match_1ro_compras_faltantes = match_1ro.loc[~match_1ro['line_id'].isin(match_1ro_compras_ok['line_id'])]

print(f'Son un total de {len(compras_fase_b)} líneas de compra en la fase B.')

print(f'   En el 1er match se emparejaron {len(match_1ro_compras_ok)} de ellas {len(match_1ro_compras_ok)/len(compras_fase_b)*100:.2f}%')
print(f'   Quedan por emparejar {len(match_1ro_compras_faltantes)} de líneas {len(match_1ro_compras_faltantes)/len(compras_fase_b)*100:.2f}%')

Son un total de 1411 líneas de compra en la fase B.
   En el 1er match se emparejaron 739 de ellas 52.37%
   Quedan por emparejar 672 de líneas 47.63%


In [64]:
# For para dar tratamiento a las líneas de compra que tuvieron tuvieron match con una misma línea de venta
wep_igual_uno = []
wep_muchas = []
wep_vacias = []

lista_ventas_merged = []

for i in range(len(match_1ro_compras_faltantes)):

    compra_df_for = match_1ro_compras_faltantes.iloc[i]

    mini_df = ventas_prod_varios_prov_after_merge1.loc[
                ~(ventas_prod_varios_prov_after_merge1['fact_line_id'].isin(lista_ventas_merged))
                & (ventas_prod_varios_prov_after_merge1['product_id'] == compra_df_for['product_id_pp'])
                & (ventas_prod_varios_prov_after_merge1['salesperson_name'] == compra_df_for['vendedora'])
                & (ventas_prod_varios_prov_after_merge1['quantity'] == compra_df_for['product_qty'])
                & (ventas_prod_varios_prov_after_merge1['invoice_date'] >= compra_df_for['order_date'] - pd.Timedelta(days=15))
                & (ventas_prod_varios_prov_after_merge1['invoice_date'] <= compra_df_for['order_date'] + pd.Timedelta(days=3))
            ]

    if not mini_df.empty:
        lista_ventas_merged.append(mini_df['fact_line_id'].iloc[0])

    if len(mini_df) == 1:
        wep_igual_uno.append(compra_df_for['line_id'])

    if len(mini_df) > 1:
        wep_muchas.append(compra_df_for['line_id'])
    
    if mini_df.empty:
        wep_vacias.append(compra_df_for['line_id'])

print(len(wep_igual_uno), 'wep_igual_uno')
print(len(wep_muchas), 'wep_muchas')
print(len(wep_vacias), 'wep_vacias')
lista_ventas_merged

NameError: name 'ventas_prod_varios_prov_after_merge1' is not defined

# No seguir  
Se ocupa terminar el costo de ventas: estas líneas son para obtener la "Utilidad" y el "Margen de Contribución".

In [ ]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [ ]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]